In [ ]:
!pip install gradio torch torchvision pillow transformers

In [ ]:
import torch
import torchvision
from PIL import Image, ImageDraw
import torchvision.transforms as T
from transformers import (
    BlipProcessor,
    BlipForConditionalGeneration,
    BlipForQuestionAnswering
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Object detection model
det_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="COCO_V1").to(device)
det_model.eval()

# Image captioning
cap_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
cap_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)
cap_model.eval()

# VQA model
vqa_processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base") # Changed to BlipProcessor
vqa_model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to(device)
vqa_model.eval()

# COCO labels
COCO_LABELS = [
    '__background__','person','bicycle','car','motorcycle','airplane','bus','train','truck','boat',
    'traffic light','fire hydrant','stop sign','parking meter','bench','bird','cat','dog','horse','sheep',
    'cow','elephant','bear','zebra','giraffe','backpack','umbrella','handbag','tie','suitcase','frisbee',
    'skis','snowboard','sports ball','kite','baseball bat','baseball glove','skateboard','surfboard',
    'tennis racket','bottle','wine glass','cup','fork','knife','spoon','bowl','banana','apple','sandwich',
    'orange','broccoli','carrot','hot dog','pizza','donut','cake','chair','couch','potted plant','bed',
    'dining table','toilet','tv','laptop','mouse','remote','keyboard','cell phone','microwave','oven',
    'toaster','sink','refrigerator','book','clock','vase','scissors','teddy bear','hair drier','toothbrush'
]

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


100%|██████████| 160M/160M [00:01<00:00, 163MB/s]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def detect_and_draw_boxes(image, threshold):
    transform = T.Compose([T.ToTensor()])
    img_tensor = transform(image).to(device)

    with torch.no_grad():
        outputs = det_model([img_tensor])[0]

    boxes = outputs["boxes"]
    labels = outputs["labels"]
    scores = outputs["scores"]

    draw = ImageDraw.Draw(image)
    detected_list = []

    for box, label, score in zip(boxes, labels, scores):
        if score >= threshold:
            x1, y1, x2, y2 = box.tolist()
            percent = int(score * 100)  # convert confidence to percentage

            # Draw bounding box
            draw.rectangle([x1, y1, x2, y2], outline="red", width=3)

            # Draw label text
            label_text = f"{COCO_LABELS[label]} — {percent}%"
            draw.text((x1, y1), label_text, fill="yellow")

            detected_list.append(label_text)

    return image, detected_list


In [ ]:
def caption_image(image):
    inputs = cap_processor(images=image, return_tensors="pt").to(device)
    out = cap_model.generate(**inputs)
    return cap_processor.decode(out[0], skip_special_tokens=True)


def answer_question(image, question):
    inputs = vqa_processor(images=image, text=question, return_tensors="pt").to(device)
    out = vqa_model.generate(**inputs)
    return vqa_processor.decode(out[0], skip_special_tokens=True)


In [ ]:
import gradio as gr

def chatbot(image, question, threshold):
    if image is None:
        return None, "Please upload an image.", "", ""

    # Draw bounding boxes
    boxed_img, detected = detect_and_draw_boxes(image.copy(), threshold)

    caption = caption_image(image)
    answer = answer_question(image, question) if question else ""

    return boxed_img, caption, detected, answer


with gr.Blocks() as demo:
    gr.Markdown("# 🖼️ Image Recognition Chatbot with Bounding Boxes + Confidence % (Gradio)")
    gr.Markdown("Upload an image → See objects → Confidence → Caption → Ask Questions")

    with gr.Row():
        img_input = gr.Image(type="pil", label="Upload Image")
        question_input = gr.Textbox(label="Ask a Question about the Image")

    # Confidence threshold slider (0–100%)
    threshold_slider = gr.Slider(0.1, 0.9, value=0.60, step=0.01, label="Confidence Threshold")

    btn = gr.Button("Analyze")

    img_output = gr.Image(label="Image with Bounding Boxes")
    caption_output = gr.Textbox(label="Caption")
    objects_output = gr.Textbox(label="Detected Objects (with %)")
    answer_output = gr.Textbox(label="Answer")

    btn.click(
        chatbot,
        inputs=[img_input, question_input, threshold_slider],
        outputs=[img_output, caption_output, objects_output, answer_output]
    )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9c0da854fa2174dbb9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
